In [1]:
!pip install tensorflow
!pip install autokeras
!pip install kaggle

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
def download_kaggle_dataset(kaggle_user:str, kaggle_key:str, data_set:str, path:str):
    import os
    import glob
   
#     print(kaggle_user)
#     print(kaggle_key)
    os.environ['KAGGLE_USERNAME'] = kaggle_user 
    os.environ['KAGGLE_KEY'] = kaggle_key

    import kaggle
    from kaggle.api.kaggle_api_extended import KaggleApi
#     os.chdir(os.environ.get('HOME'))
    os.chdir(os.getcwd())
    os.system("mkdir " + path)
    os.chdir(path)
    api = KaggleApi()
    api.authenticate()
    api.competition_download_file('facial-keypoints-detection','training.zip')
    api.competition_download_file('facial-keypoints-detection','test.zip')


print('downloading data fom Kaggle ... ')
download_kaggle_dataset("username", "key", "facial-keypoints-detection","my_data")



downloading data fom Kaggle ... 


100%|██████████| 60.1M/60.1M [00:00<00:00, 167MB/s] 


100%|██████████| 16.0M/16.0M [00:00<00:00, 129MB/s] 

In [1]:
import numpy as np
import os
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import tensorflow as tf                
import pandas as pd
from tensorflow.keras.models import load_model
import os
import shutil
import argparse
import autokeras as ak
from tensorflow.keras.models import load_model
import autokeras as ak
import pandas as pd
import numpy as np


In [ ]:
trials = 1
epochs = 1
patience = 1

In [5]:


project="Facial-keypoints"
run_id= "1.8"
resume_run = True

MAX_TRIALS=int(trials)
EPOCHS=int(epochs)
PATIENCE=int(patience)

### Data Extraction : extract data and save to attached extenal pvc at location /data ###

base_dir='my_data/'
train_dir_zip=base_dir+'training.zip'
test_dir_zip=base_dir+'test.zip'

from zipfile import ZipFile
with ZipFile(train_dir_zip,'r') as zipObj:
    zipObj.extractall('data')
    print("Train Archive unzipped")
with ZipFile(test_dir_zip,'r') as zipObj:
    zipObj.extractall('data')
    print("Test Archive unzipped")


## Data preprocess 

train_dir='data/training.csv'
test_dir='data/test.csv'
train=pd.read_csv(train_dir)
test=pd.read_csv(test_dir)

train=train.dropna()
train=train.reset_index(drop=True)

X_train=[]
Y_train=[]

for img in train['Image']:
    X_train.append(np.asarray(img.split(),dtype=float).reshape(96,96,1))
X_train=np.reshape(X_train,(-1,96,96,1))
X_train = np.asarray(X_train).astype('float32')
    
for i in range(len((train))): 
    Y_train.append(np.asarray(train.iloc[i][0:30].to_numpy()))
Y_train = np.asarray(Y_train).astype('float32')

## Data training

reg = ak.ImageRegressor(max_trials=MAX_TRIALS)
reg.fit(X_train, Y_train, validation_split=0.15, epochs=EPOCHS)

# Export trained model to externally attached pvc 
my_model = reg.export_model()
my_model.save('data/model_autokeras', save_format="tf")


Train Archive unzipped
Test Archive unzipped
INFO:tensorflow:Reloading Oracle from existing project ./image_regressor/oracle.json
INFO:tensorflow:Reloading Tuner from ./image_regressor/tuner0.json
INFO:tensorflow:Oracle triggered exit
67/67 [==============================] - 187s 3s/step - loss: 1707.4453 - mean_squared_error: 1707.4453


/home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: ./image_regressor/best_model/assets


In [3]:

### Load model 
loaded_model = load_model("data/model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)

### Pint model summary
print(loaded_model.summary())

test_dir='data/test.csv'
test=pd.read_csv(test_dir)

X_test=[]
for img in test['Image']:
    X_test.append(np.asarray(img.split(),dtype=float).reshape(96,96,1))
X_test=np.reshape(X_test,(-1,96,96,1))
X_test = np.asarray(X_test).astype('float32')

### predict 
y_pred = loaded_model.predict(X_test)

### Create submission file
y_pred= y_pred.reshape(-1,)
submission = pd.DataFrame({'Location': y_pred})
submission.to_csv('data/submission.csv', index=True , index_label='RowId')
